In [ ]:
!pip install deepface wandb tensorflow-addons keras huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=c9dcb8567fd1748506430184eb2e8714d14488110af339d307cdc36ef2204b28
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8

# Downloading dataset

In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2024-07-10 12:09:54--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  2.43MB/s    in 2m 13s  

2024-07-10 12:12:07 (1.30 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
!tar -xzvf lfw.tgz

write code to clean the dataset where if any folder in lfw/ has less than 4 images then delete the directory

In [ ]:
import os
import shutil
count=0
for folder in os.listdir('lfw'):
    if len(os.listdir(f'lfw/{folder}')) < 5:
        shutil.rmtree(f'lfw/{folder}')
        count+=1
print(f"Removed {count} folders")

Removed 3 folders


In [ ]:
print(len(os.listdir('lfw')))

5


Import Modules

In [ ]:
import deepface
from deepface.basemodels.Facenet import load_facenet512d_model
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

In [ ]:
def get_embedding_module(imageSize):

    # Load Facenet Model
    model = load_facenet512d_model()

    inputs = keras.Input(shape=(imageSize, imageSize, 3))

    # Resizing the input
    x = layers.Resizing(160, 160)(inputs)
    output = model(x)
    embedding = keras.Model(inputs, output, name="embedding")
    return embedding

In [ ]:
import os

directory = '/root/.deepface/weights'
os.makedirs(directory, exist_ok=True)

In [ ]:
def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
class TripletDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size, num_classes):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(labels)
        self.image_data_generator = ImageDataGenerator()
        self.on_epoch_end()
        print(f"Initialized TripletDataGenerator with {len(self.image_paths)} images")

    def __len__(self):
        return max(1, len(self.image_paths) // self.batch_size)  # Ensure at least one batch

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.encoded_labels[index * self.batch_size:(index + 1) * self.batch_size]
        return self._generate_triplet_batch(batch_image_paths, batch_labels)

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        combined = list(zip(self.image_paths, self.encoded_labels))
        np.random.shuffle(combined)
        self.image_paths[:], self.encoded_labels[:] = zip(*combined)

    def _generate_triplet_batch(self, batch_image_paths, batch_labels):
        anchor_images = []
        positive_images = []
        negative_images = []

        for i in range(len(batch_image_paths)):
            anchor_path = batch_image_paths[i]
            anchor_label = batch_labels[i]

            positive_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l == anchor_label]
            )
            negative_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l != anchor_label]
            )

            anchor_image = load_img(anchor_path, target_size=self.image_size)
            positive_image = load_img(positive_path, target_size=self.image_size)
            negative_image = load_img(negative_path, target_size=self.image_size)

            anchor_images.append(img_to_array(anchor_image))
            positive_images.append(img_to_array(positive_image))
            negative_images.append(img_to_array(negative_image))

        return (
            {
                "anchor": np.array(anchor_images),
                "positive": np.array(positive_images),
                "negative": np.array(negative_images)
            },
            None,
        )

In [ ]:
class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker

    def _compute_distance(self, inputs):
        anchor, positive, negative = inputs["anchor"], inputs["positive"], inputs["negative"]
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )
        # return the distances
        return (apDistance, anDistance)

    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        return (apDistance, anDistance)

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            apDistance, anDistance = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    @property
    def metrics(self):
        return [self.lossTracker]

    def get_config(self):
        config = super().get_config()
        config.update({
            "siameseNetwork": self.siameseNetwork,
            "margin": self.margin,
            "lossTracker": self.lossTracker
            })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Try-1

In [ ]:
# Set the directory structure
data_dir = 'lfw'
image_size = (250, 250)
batch_size = 32  # Adjust the batch size for the small dataset
margin = 1.0

In [ ]:
# Initialize W&B
wandb.init(project="FaceRec", config={
    "learning_rate": 0.0001,
    "epochs": 20,
    "batch_size": 32,
    "optimizer": "Adam",
    "architecture": "FaceNet",
    "dataset": "lfw",
    "loss": "TripletLoss",
    "margin": 1.0
})

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Load and preprocess the data
image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)

In [ ]:
# Debugging output
print(f"Found {len(image_paths)} images in {len(set(labels))} classes")

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Check if the splits are non-empty
print(f"Training on {len(train_paths)} images")
print(f"Validating on {len(val_paths)} images")

Found 1140 images in 5 classes
Training on 912 images
Validating on 228 images


In [ ]:
# Create data generators
num_classes = len(set(labels))
train_generator = TripletDataGenerator(train_paths, train_labels, batch_size, image_size, num_classes)
val_generator = TripletDataGenerator(val_paths, val_labels, batch_size, image_size, num_classes)

# Check if the generators have data
assert len(train_generator) > 0, "Training generator is empty!"
assert len(val_generator) > 0, "Validation generator is empty!"

Initialized TripletDataGenerator with 912 images
Initialized TripletDataGenerator with 228 images


In [ ]:
# Create the embedding model and the Siamese network
embedding_model = get_embedding_module(image_size[0])
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam())

24-07-10 12:14:58 - facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:00<00:00, 97.0MB/s]


In [ ]:
# Train the model
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models")]
)

Epoch 1/2
114/114 [==============================] - ETA: 0s - loss: 62.7843

wandb: Adding directory to artifact (./models)... Done. 1.6s


114/114 [==============================] - 747s 7s/step - loss: 62.7843 - val_loss: 1.0428
Epoch 2/2
114/114 [==============================] - ETA: 0s - loss: 1.0216

wandb: Adding directory to artifact (./models)... Done. 6.7s


114/114 [==============================] - 745s 7s/step - loss: 1.0216 - val_loss: 0.9650


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/content/models",
    repo_id="DShah-11/FaceNet_Finetuned",
    repo_type="model",
)

AttributeError: 'SiameseModel' object has no attribute 'push_to_hub'